# Ray Serve | Ray Serve[Ray Serve](https://docs.ray.io/en/latest/serve/index.html)は、オンライン推論APIを構築するためのスケーラブルなモデル提供ライブラリです。Serveは特にシステム構成に適しており、Pythonコードだけで複数の連鎖とビジネスロジックを含む複雑な推論サービスを構築することができます。> [Ray Serve](https://docs.ray.io/en/latest/serve/index.html) is a scalable model serving library for building online inference APIs. Serve is particularly well suited for system composition, enabling you to build a complex inference service consisting of multiple chains and business logic all in Python code.

## Goal of this notebook | このノートブックの目的このノートブックは、OpenAIチェーンを本番環境にデプロイする簡単な例を示しています。自己ホスト型のモデルをデプロイする際に、本番環境でモデルを効率的に実行するために必要なハードウェアリソース（GPUとCPU）の量を簡単に定義できるよう拡張することができます。オートスケーリングを含む利用可能なオプションについては、Ray Serveの[ドキュメント](https://docs.ray.io/en/latest/serve/getting_started.html)をご覧ください。> This notebook shows a simple example of how to deploy an OpenAI chain into production. You can extend it to deploy your own self-hosted models where you can easily define amount of hardware resources (GPUs and CPUs) needed to run your model in production efficiently. Read more about available options including autoscaling in the Ray Serve [documentation](https://docs.ray.io/en/latest/serve/getting_started.html).

## Setup Ray Serve | Ray Serveのセットアップ`pip install ray[serve]`を使用してrayをインストールしてください。> Install ray with `pip install ray[serve]`.

## General Skeleton | 一般的な骨格

サービスをデプロイするための一般的なスケルトンは以下の通りです：> The general skeleton for deploying a service is the following:

In [ ]:
# 0: Import ray serve and request from starlette
from ray import serve
from starlette.requests import Request


# 1: Define a Ray Serve deployment.
@serve.deployment
class LLMServe:
    def __init__(self) -> None:
        # All the initialization code goes here
        pass

    async def __call__(self, request: Request) -> str:
        # You can parse the request here
        # and return a response
        return "Hello World"


# 2: Bind the model to deployment
deployment = LLMServe.bind()

# 3: Run the deployment
serve.api.run(deployment)

In [ ]:
# Shutdown the deployment
serve.api.shutdown()

## Example of deploying and OpenAI chain with custom prompts | カスタムプロンプトを使用してOpenAIチェーンをデプロイする例

[こちら](https://platform.openai.com/account/api-keys)からOpenAIのAPIキーを取得してください。以下のコードを実行すると、APIキーの入力を求められます。> Get an OpenAI API key from [here](https://platform.openai.com/account/api-keys). By running the following code, you will be asked to provide your API key.

In [ ]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [ ]:
from getpass import getpass

OPENAI_API_KEY = getpass()

In [ ]:
@serve.deployment
class DeployLLM:
    def __init__(self):
        # We initialize the LLM, template and the chain here
        llm = OpenAI(openai_api_key=OPENAI_API_KEY)
        template = "Question: {question}\n\nAnswer: Let's think step by step."
        prompt = PromptTemplate(template=template, input_variables=["question"])
        self.chain = LLMChain(llm=llm, prompt=prompt)

    def _run_chain(self, text: str):
        return self.chain(text)

    async def __call__(self, request: Request):
        # 1. Parse the request
        text = request.query_params["text"]
        # 2. Run the chain
        resp = self._run_chain(text)
        # 3. Return the response
        return resp["text"]

これでデプロイメントをバインドすることができます。> Now we can bind the deployment.

In [ ]:
# Bind the model to deployment
deployment = DeployLLM.bind()

デプロイメントを実行する際には、ポート番号とホストを指定することができます。> We can assign the port number and host when we want to run the deployment.

In [ ]:
# Example port number
PORT_NUMBER = 8282
# Run the deployment
serve.api.run(deployment, port=PORT_NUMBER)

サービスがポート`localhost:8282`にデプロイされたので、POSTリクエストを送信して結果を取得することができます。> Now that service is deployed on port `localhost:8282` we can send a post request to get the results back.

In [ ]:
import requests

text = "What NFL team won the Super Bowl in the year Justin Beiber was born?"
response = requests.post(f"http://localhost:{PORT_NUMBER}/?text={text}")
print(response.content.decode())